# Introduction

This script is used to gather the necessary data.
Things to be fixed:
* Token still needs to be refreshed by hand
* Choose time range yourself by inserting the right parameters
* Choose things you want to investigate
* Choose metrics of interest

# Initialization

## Import libraries

In [4]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json
from datetime import datetime
from datetime import timedelta  
import time
import sys
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import os

pd.options.mode.chained_assignment = None 

## Set up parameters

In [5]:
##### Choose scope #####
scopeID = 'cot.smart_lighting'

In [6]:
##### Insert token and generate URL #####
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiJhODgyMDAwZS04MWYwLTQ2MjAtODgxNy05YjI2ZjZiNjY1YWQiLCJleHAiOjE1NTQyMjY0NjQsIm5iZiI6MCwiaWF0IjoxNTU0MjI1ODY0LCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiIyMjNkNjQ3My0wZmU3LTQxNmItODNiNS0wN2I3NmRjYTlkOTgiLCJhdXRoX3RpbWUiOjE1NTQyMjU4NjMsInNlc3Npb25fc3RhdGUiOiIzMjBkZDkzZS1kZDEyLTRmNzktYjA0ZC0zZTlhNDlkMmNiNGMiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.nhpHfml-_HOvvc2r3y5VCl0xPBz33_kcB4cn1iDSdGF-dkwi6lkSPSTsNxltxU9d8ZMtIviMcU0ha4dt87NJ7RZZRCFUcmlR5GjKuD1qA5NbmvoTSDums8_kAWj-LrBFQqsFxa8TwoQK5sLLMYTDbBt8CYtiNGaZDUifEg9t_yc8Mck3N2QBG-G28_H4zhDPmQvEP0Kg_X8V7nn5a8f2Qeh20XSKcMUxLn80oHXfaC5O0P-CS2OA_TeKaN0UQdz6EjkKtQlsIiKRAZ8Syevcl77x3ZfWyTnlcQ0-BKKD8nj5TVf55eSzVID6HZnt-9_LVBBOB4CEnm35U-zMh8exFA'

In [7]:
##### Specify time range #####
##### Adjust this time range to specify which period you want to query - Mostly I use periods of one month #####
startDate = datetime(2019, 1, 1, 1, 0)
endDate = datetime(2019, 2, 1, 1, 0)
startTime = int(startDate.timestamp())*1000;  #print(startTime)
endTime = int(endDate.timestamp())*1000;  #print(endTime)

# Weather station

In [5]:
thingID = 'davis.davis.weather.1'

### Choose metrics

In [6]:
metricsList = ['environment.light','environment.temperature','weather.uv','environment.relativehumidity','weather.pressure', 'weather.rainrate', 'weather.windspeed']
# metricsList = ['weather.pressure', 'weather.rainrate', 'weather.windspeed']

### Run query

In [7]:
for i in range(0,len(metricsList)):
    print(metricsList[i]+'_'+startDate.strftime('%Y-%m'))
    metricID = metricsList[i]

    myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    response = requests.get(myUrlQuery, headers=headers)
    #print(response.content)
    data = response.json() 
    column_names = data['columns']; #print(column_names)
    values = data['values']
    df = pd.DataFrame.from_dict(values)
    df.columns = column_names
    weather_data = df[['time','value']]
#     print(weather_data)
#     path=r'C:\Users\JeffG\Desktop\Case 1 - data\weather_data'
    path=r'D:\Documents\Thesis\Case 1 - data\weather_data'
    weather_data.to_csv(path+'\\'+metricsList[i]+'_'+startDate.strftime('%Y-%m')+'.csv')

environment.light_2019-03
environment.temperature_2019-03
weather.uv_2019-03
environment.relativehumidity_2019-03
weather.pressure_2019-03
weather.rainrate_2019-03
weather.windspeed_2019-03


# Battery pack

In [11]:
#thingID = 'munisense.msup1g30034' # west facing
thingID = 'munisense.msup1i70124' # south facing
# thingID = 'munisense.msup1h90115' # east facing

### Choose metrics

In [12]:
# metricsList = ['Power.BatteryState']
metricsList = ['Power.BatteryState','Power.BatteryVoltHR','Power.PercentageRemaining','Power.TimeRemaining']

### Run query

In [13]:
for i in range(0,len(metricsList)):
    print(metricsList[i]+'_'+startDate.strftime('%Y-%m'))
    metricID = metricsList[i]

    myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    response = requests.get(myUrlQuery, headers=headers)
    response.content
    data = response.json(); #print(data)
    column_names = data['columns']; #print(column_names)
    values = data['values']
    df = pd.DataFrame.from_dict(values)
    df.columns = column_names
    battery_data = df[['time','value']]
#     print(battery_data)
    path=r'D:\Documents\Thesis\Case 1 - data\battery_data_2' #vaste pc
#     path=r'C:\Users\JeffG\Desktop\Case 1 - data\battery_data_2' #Laptop
    battery_data.to_csv(path+'\\'+metricsList[i]+'_'+startDate.strftime('%Y-%m')+'.csv')

Power.BatteryState_2019-03
Power.BatteryVoltHR_2019-03
Power.PercentageRemaining_2019-03
Power.TimeRemaining_2019-03


# General functions

In [8]:
def load_data_to_csv(scopeID, listOfThings, listOfMetrics, startDate, endDate, myToken, basePath):
    """
    This function will generate a local .csv-file per chosen Thing and Metric for a certain period.

    Args:
        scopeID: The ID of the scope as defined on the IoT-stack explorer
        listOfthings: List of the things you want to query as defined on the IoT-stack explorer
        listOfMetrics: List of metrics you want to query as defined on the IoT-stack explorer
        startDate: Earliest date in datetime format
        endDate: Latest date in datetime format
        myToken: Token generated through Swagger UI

    Returns:
        Nothing. Saves the .csv-file locally or prints an error message

    ValueError:
        ValueError: Raises an exception.
    """
    #Convert timestamp to milliseconds
    startTime = int(startDate.timestamp())*1000
    endTime = int(endDate.timestamp())*1000
    
    for thingID in listOfThings:
        for metricID in listOfMetrics:
            # Specify the right API call
            myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
                .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

            headers = {
                'accept': 'application/json',
                'authorization': 'Bearer {0}'.format(myToken),
            }
            # Execure query
            response = requests.get(myUrlQuery, headers=headers)
            try:
                # Transform to json format
                data = response.json()
                if len(data)!=0:
                    # Get columns names and values
                    column_names = data['columns']; #print(column_names)
                    values = data['values']
                    # Get data in pandas DataFrame format
                    df = pd.DataFrame.from_dict(values)
                    df.columns = column_names
                    result = df[['time','value']]
                    # Set path
                    if thingID == 'davis.davis.weather.1':
                        path=basePath +'\\weather_data'
                        if not os.path.exists(path):
                            os.mkdir(path)
                            print("Directory " , path ,  " Created ")
                    else:
                        path=basePath+ '\\battery_data'
                        if not os.path.exists(path):
                            os.mkdir(path)
                            print("Directory " , path ,  " Created ")
                        path=path + '\\' + str(thingID)
                        if not os.path.exists(path):
                            os.mkdir(path)
                            print("Directory " , path ,  " Created ")
                    # Save DataFrame to a local file
                    result.to_csv(path+'\\'+str(metricID)+'_'+startDate.strftime('%Y-%m')+'.csv')

            except ValueError:  
                print("Decoding JSON has failed. Probably the token has expired")
                print(ValueError)

In [53]:
scopeID = 'cot.smart_lighting'
basePath = r'D:\Documents\Thesis\Case 1 - data'
listOfThings = ['munisense.msup1g30034', 'munisense.msup1i70124', 'munisense.msup1h90115', 'munisense.msup1h90103']
listOfMetrics = ['Power.BatteryState','Power.BatteryVoltHR','Power.PercentageRemaining','Power.TimeRemaining']
startDate = datetime(2018, 8, 1, 1, 0)
endDate = datetime(2018, 9, 1, 1, 0)
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiI3ODg0MmVmZi03Zjg1LTQ4ZGEtOGFmNC05Y2JjNWIxZWIzNDciLCJleHAiOjE1NTQzNjc3MTcsIm5iZiI6MCwiaWF0IjoxNTU0MzY3MTE3LCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiIxZDIzZjQwYi02ZTQ2LTQ2NjYtOGE2OS1mY2I5YThjYWJkMmUiLCJhdXRoX3RpbWUiOjE1NTQzNjU2OTMsInNlc3Npb25fc3RhdGUiOiJlYTExZTRhZi0wNjRiLTRjOTktOTE4ZC03NjUxMTFlZmNkYmEiLCJhY3IiOiIwIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.BxSqS-65w-8de0pQxE84Hu5wJMH29KithUbRav07nAkT8U8dOVUx8wiFofWGU_wEQJXxtmjZodHgmhQU7Hfd4Oh4wYLbpY5o12ViB7Dkbt-r6nOx0aBqcK0IekHWTI9Q8pjOVE-wldTIv34SKrk8MfPcleGCUE8c1yxGoYZVhVGudn0wMk4C4kg9BsdEEmLwYus5lpiQLar9UVf9aH3S7IPcumWgy3Xumgx9YsfPQRdPZp6jswsonhruu4A9MTRjf90XlYNW217Lqu6MYDiexHjuBH20V0JWlbyXtsNlQ_VqdrZU2rjfLFi1tqcHEwYC3cxj1XVj6175uEq-aY2uuA'

In [54]:
load_data_to_csv(scopeID, listOfThings, listOfMetrics, startDate, endDate, myToken, basePath)

In [9]:
def load_voltage(startDate, endDate, thingID, myToken):
    ##### Specify time range #####
    ##### Adjust this time range to specify which period you want to query - Mostly I use periods of one month #####
    startTime = int(startDate.timestamp())*1000;  #print(startTime)
    endTime = int(endDate.timestamp())*1000;  #print(endTime)

    ##### Choose scope #####
    scopeID = 'cot.smart_lighting'

    ##### Specify query #####
    metricID = 'Power.BatteryVoltHR'
    myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    ##### Execute query #####
    response = requests.get(myUrlQuery, headers=headers)

    try:
        data = response.json(); 
        column_names = data['columns']; 
        values = data['values']
        df = pd.DataFrame.from_dict(values)
        df.columns = column_names
        df["value"] = pd.to_numeric(df.value, errors='coerce')

        ##### Clean up dataFrame #####
        # Rename the column
        df.rename(columns={'value':'Power.BatteryVoltHR'}, inplace=True)
        # Set timestamp as index
        df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.time, unit='ms')))
        df.drop(['time'], axis=1,inplace=True)
        message = 'All good'

        return df, message

    except ValueError:
        df = None
        errorMessage = "Decoding JSON has failed. Probably the token has expired"
        return df, errorMessage
    

In [ ]:
endDate = datetime.now()
startDate = endDate - timedelta(3/12*365)
thingID = 'munisense.msup1h90115' # east facing
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiI1Y2Y3NjAyNi1mYTJhLTRmZTYtYWU5ZC02NDU1MjI3ZWVhZDciLCJleHAiOjE1NTQxNDc0MDQsIm5iZiI6MCwiaWF0IjoxNTU0MTQ2ODA0LCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiJhMzRmMmE1Yy02NzgzLTRmYmMtOTNkMy1kOWIyNDhhNTFmZmEiLCJhdXRoX3RpbWUiOjE1NTQxNDY4MDMsInNlc3Npb25fc3RhdGUiOiIwOTQyN2VkMC02YTg0LTRlZDktOTYyOC04NzkyMjAzZTM2ODkiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.hz9g4pUg-_xFRWum7BR4hhjCW3mQEaBgQ3VfJe-IRoDoIa6FmDQ7J-AQ7W4PbkSr1cXw4sy94B4fL_6GYoV2QwdqJNEqipCV0YVvfNt7UmLeyLMLDPQbZIrRMu-O1k-JQ67nroFlPCQKrIdB5pvfqYHQzpEkp6niAQu_Mypm2Hgz7ulSC0xlgY3rqmkIo1VhpRNULEmS1B19_91DdFgmADX6843HtOlJCll4xJ-hLfpnFBNejczNX8gQ9ImfRzJxSYEAAwETQLVgDXu_gQyeo7z4P0OuWx51SDAhhHFp9f5tfjLDvb86Kk2sZxSbdsH9sMANAfzduMGjFVfZIzxOFA'

df = load_voltage(startDate, endDate, thingID, myToken)

In [ ]:
df['moving_average'] = df['Power.BatteryVoltHR'].rolling(window_size).mean()
df['residual'] = df['Power.BatteryVoltHR'] - df['moving_average']

# Label anomalies

In [43]:
def label_anomaly_moving_std (row, sigma):
    if (row['Power.BatteryVoltHR'] > (row['moving_average'] + (sigma * row['moving_std']))) | (row['Power.BatteryVoltHR'] < (row['moving_average'] - (sigma * row['moving_std']))):
        return 1
    else:
        return 0

def label_anomaly_fixed_std (row, sigma, fixed_std):
    if (row['Power.BatteryVoltHR'] > (row['moving_average'] + (sigma * fixed_std))) | (row['Power.BatteryVoltHR'] < (row['moving_average'] - (sigma * fixed_std))):
        return 1
    else:
        return 0

# Low pass filtering

In [42]:
def low_pass_filtering(df, window_size, sigma, fixed_std):
    # Calculate moving average
    df['moving_average'] = df['Power.BatteryVoltHR'].rolling(window_size).mean()

    # Calculate anomalies
    df['residual'] = df['Power.BatteryVoltHR'] - df['moving_average']
    df['moving_std'] = df['Power.BatteryVoltHR'].rolling(window_size).std()
    df.dropna(inplace=True)
    df['anomaly_flag_moving_std']=df.apply (lambda row: label_anomaly_moving_std(row, sigma), axis=1)
    df['anomaly_flag_fixed_std']=df.apply (lambda row: label_anomaly_fixed_std(row, sigma, fixed_std), axis=1)

    moving_anomalies = df[df['anomaly_flag_moving_std']==1]
    fixed_anomalies = df[df['anomaly_flag_fixed_std']==1]
    
    return df

In [ ]:
df = low_pass_filtering(df, 256, 3, fixed_std)
moving_anomalies = df[df['anomaly_flag_moving_std']==1]
fixed_anomalies = df[df['anomaly_flag_fixed_std']==1]

# Saving data to SQL database

## Load libraries

In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

In [2]:
def initialize_database():
    user = 'root'
    passw = ''
    host =  'localhost'  # either localhost or ip e.g. '172.17.0.2' or hostname address 
    port = 3306 
    database = 'smart_lighting'
    engine = create_engine('mysql+mysqlconnector://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database , echo=False)
    return engine

def load_voltage_from_database(thingID, startDate, endDate, engine):
    table_name = 'battery_voltage'
    start_date = startDate.strftime("%Y-%m-%d %H:%M:%S")
    end_date = endDate.strftime("%Y-%m-%d %H:%M:%S")
    query = ''' SELECT * 
                FROM {} 
                WHERE (sourceId = '{}') AND (time between '{}' and '{}')'''.format(table_name, thingID, start_date, end_date)
    df = pd.read_sql(query, engine, index_col='time')
    return df

In [13]:
endDate = datetime.now()
startDate = endDate - timedelta(9/12*365)
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiI2MDliYTAyYi1iOGFkLTQ5OGYtOWQ1My1mOTVmODE0ZjM4OWMiLCJleHAiOjE1NTQ4ODMzNzYsIm5iZiI6MCwiaWF0IjoxNTU0ODgyNzc2LCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiJjZTU1YWEyYS1mNjdkLTQyNDgtYThmZS05OGYzNWI3NzE3NTYiLCJhdXRoX3RpbWUiOjE1NTQ4ODIwMjMsInNlc3Npb25fc3RhdGUiOiI4ZjZhOGQ1Zi1jZDk2LTRiNDQtYTY0MC01NjgyYjI1MTJiZTEiLCJhY3IiOiIwIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.EpqD5D00hG952FauB5XaEGobFhT8PUvkI5MRHmFF5Eg65HQaTsKcHeY8RAxxGvlVXWf7yy8o8c4Doir4bOirl4ZLyVxYOTNCz5Dd2NTwNPWqtX80pxOQyftpHVOM1vMzvmKKi6B2DaoW39ejNg97I0kDuACjgKHEiH9Dnj7rygzNfhwhZ5vM2jDQcTLrMUSQDyvb2Q9ALbOLFRIMSuEKh5pK1flAicufMapDI1tETP8shjmj9nD-Z3wcOBBEdUzO4msd4trjAHctEjULHACKrwKypkMyxrf0pFpBItAtvBdn4shQKMjMkBgo_W9bhO-YE_iDeY5auDnHYysSrL398A'
things = ['munisense.msup1g30034', 'munisense.msup1i70124', 'munisense.msup1h90115', 'munisense.msup1h90103']
engine = initialize_database()
for thingID in things:
    df,message = load_voltage(startDate, endDate, thingID, myToken)
    df.to_sql(name='battery_voltage', con=engine, if_exists = 'append', index=True)

In [20]:
endDate = datetime.now(); print(endDate)
startDate = endDate - timedelta(8/12*365); print(startDate)
df1 = load_voltage_from_database(thingID, startDate, endDate, engine)
startDate = find_last_datapoint(thingID)
df2 = load_voltage_from_database('munisense.msup1h90103', startDate, endDate, engine)


2019-04-10 10:12:44.990227
2018-08-10 02:12:44.990227


NameError: name 'find_last_datapoint' is not defined

# Query all battery voltages and save them in one DataFrame

In [22]:
def load_data_on_startup(myToken):
    endDate = datetime.now()
    startDate = datetime(2018, 8, 1, 1, 0)
    things = ['munisense.msup1g30034', 'munisense.msup1i70124', 'munisense.msup1h90115', 'munisense.msup1h90103']
    battery_voltages = pd.DataFrame()
    for thingID in things:
        df,message = load_voltage(startDate, endDate, thingID, myToken)
        battery_voltages = battery_voltages.append(df)

    return battery_voltages

In [24]:
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiIxODRmOGQ2MS05NzliLTQ0N2ItODZhYi1hYmQ1ZTM1MGJlYWMiLCJleHAiOjE1NTQ4ODQ0MjgsIm5iZiI6MCwiaWF0IjoxNTU0ODgzODI4LCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiIyZmYwYTAwYS0xZTcyLTRlZDMtOGNkNi00MTQ1YWQ3YWRhNGEiLCJhdXRoX3RpbWUiOjE1NTQ4ODIwMjMsInNlc3Npb25fc3RhdGUiOiI4ZjZhOGQ1Zi1jZDk2LTRiNDQtYTY0MC01NjgyYjI1MTJiZTEiLCJhY3IiOiIwIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.nlymYprtSynSKsNFaQVM7-KzII7HzdKgEbwiCt6BMlMKlnGGu5Y7bh0BWuALwgWlJosHWpuYvT0y3gLEPDMzNAFAsCix6ODCDusj9RuRKR4jheU7ZSuaejyLZGVBnRAhPRqrE9Reb8d0GBOK-y9u_E5AD92m_Dl3OcQIaej3e-S2SDfRBWBTXCG8smUCoRZtmsrnC4VJKdkuND6VWLl5rgGOo4iuKZcLxRSA9clPpQ40q6weuPx_KdHFInqTtnMKrT9MnYrtRfauhpWoe6-IkOyACRRoniCKwycZH3FkO6nTRA3Pp0Uy9apVmqTRcN6Da6GNKGn36vSnxE_fs5fYow'
df = load_data_on_startup(myToken)

In [32]:
df[df['sourceId']=='munisense.msup1g30034'].iloc[-1].name

Timestamp('2019-01-28 15:57:36.923000')

In [33]:
def find_last_datapoint(thingID, df):
    result = df[df['sourceId']==thingID].iloc[-1]
    return result.name

def find_first_datapoint(thingID):
    result = df[df['sourceId']==thingID].iloc[0]
    return result.name

In [37]:
thingID = 'munisense.msup1g30034'
startDate = datetime(2018, 9, 1, 1, 0)
endDate = datetime(2018, 11, 1, 1, 0)
df[(df['sourceId']=='munisense.msup1g30034') & (df.index<endDate) & (df.index>startDate)]

,geohash,sourceId,Power.BatteryVoltHR
time,,,
2018-09-01 01:02:16.723,u155k52gzz91,munisense.msup1g30034,16.01
2018-09-01 01:05:16.723,u155k52gzz91,munisense.msup1g30034,16.01
2018-09-01 01:08:16.687,u155k52gzz91,munisense.msup1g30034,16.01
2018-09-01 01:11:16.730,u155k52gzz91,munisense.msup1g30034,16.00
2018-09-01 01:14:16.805,u155k52gzz91,munisense.msup1g30034,16.00
2018-09-01 01:17:16.799,u155k52gzz91,munisense.msup1g30034,16.01
2018-09-01 01:20:16.779,u155k52gzz91,munisense.msup1g30034,16.01
2018-09-01 01:23:16.740,u155k52gzz91,munisense.msup1g30034,16.00
2018-09-01 01:26:16.734,u155k52gzz91,munisense.msup1g30034,16.00


# Run anomaly detection on all battery packs

In [ ]:
things = ['munisense.msup1i70124', 'munisense.msup1h90115', 'munisense.msup1h90103']
for thingID in things:
    endDate = datetime.now()
    startDate = endDate - timedelta(1/12*365)
    df = load_voltage_from_database(thingID, startDate, endDate, engine)
    df = low_pass_filtering(df, window_size, sigma, fixed_std)

In [44]:
thingID = 'munisense.msup1i70124'
endDate = datetime.now()
startDate = endDate - timedelta(1/12*365)
df = load_voltage_from_database(thingID, startDate, endDate, engine)
df = low_pass_filtering(df, 256, 3, 0.3)

In [48]:
startDate = endDate - timedelta(7)
intermediate= df[(df['sourceId']==thingID) & (df.index<endDate) & (df.index>startDate)]
intermediate.head()

,geohash,sourceId,Power.BatteryVoltHR,moving_average,residual,moving_std,anomaly_flag_moving_std,anomaly_flag_fixed_std
time,,,,,,,,
2019-04-03 13:31:02,u155k52gzz91,munisense.msup1i70124,16.17,16.189219,-0.019219,0.026020,0,0
2019-04-03 13:34:02,u155k52gzz91,munisense.msup1i70124,16.18,16.189023,-0.009023,0.025900,0,0
2019-04-03 13:37:02,u155k52gzz91,munisense.msup1i70124,16.18,16.188828,-0.008828,0.025778,0,0
2019-04-03 13:46:02,u155k52gzz91,munisense.msup1i70124,16.17,16.188594,-0.018594,0.025675,0,0
2019-04-03 13:52:02,u155k52gzz91,munisense.msup1i70124,16.18,16.188398,-0.008398,0.025548,0,0


In [52]:
intermediate[intermediate['anomaly_flag_fixed_std']==0].iloc[-1].name

Timestamp('2019-04-10 09:22:03')

In [50]:
sum(intermediate['anomaly_flag_fixed_std'])

0